In [4]:
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import to_timestamp, col

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/11 09:40:32 WARN Utils: Your hostname, DESKTOP-FJ6N83N resolves to a loopback address: 127.0.1.1; using 172.20.58.135 instead (on interface eth0)
24/06/11 09:40:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/11 09:40:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
def load_config(spark_context: SparkContext):
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.access.key", os.environ['MINIO_ACCESS_KEY'])
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.secret.key", os.environ['MINIO_SECRET_KEY'])
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.endpoint", os.environ['MINIO_ENDPOINT'])
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")

load_config(spark.sparkContext)

In [ ]:
dataframe = spark.read.csv("s3a://diabetes/*", header=True, inferSchema=True)

In [2]:
aws_access_key_id = os.environ.get('MINIO_ACCESS_KEY')
aws_secret_key_id = os.environ.get('MINIO_SECRET_KEY')
aws_endpoint = os.environ.get('MINIO_ENDPOINT')
bucket = os.environ.get('MINIO_BUCKET')
print(aws_access_key_id, aws_access_key_id, aws_endpoint)

minio_access_key minio_access_key http://localhost:9001


In [3]:
conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
        .set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.4')
        .set('spark.hadoop.fs.s3a.fast.upload', 'true')
        .set('spark.hadoop.fs.s3a.path.style.access', 'true')
        .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')  
        .set('spark.hadoop.fs.s3a.endpoint', aws_endpoint)
        .set('spark.hadoop.fs.s3a.access.key', aws_access_key_id)
        .set('spark.hadoop.fs.s3a.secret.key', aws_secret_key_id)
        .set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
)

spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark running")

your 131072x1 screen size is bogus. expect trouble
24/06/10 20:23:52 WARN Utils: Your hostname, DESKTOP-FJ6N83N resolves to a loopback address: 127.0.1.1; using 172.20.58.135 instead (on interface eth0)
24/06/10 20:23:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/nampq/.cache/pypoetry/virtualenvs/feast-4zXjf9Ge-py3.9/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/nampq/.ivy2/cache
The jars for the packages stored in: /home/nampq/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-921a572c-92b6-4ee7-9d94-4d9dc950488c;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
:: resolution report :: resolve 178ms :: artifacts dl 7ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

Spark running


In [4]:
data_id = '/home/nampq/projects/feature-store/batch_processing/test.csv'

data = spark.read.format('csv').option('header', 'true').load(data_id)

In [5]:
data.show()

+---+-------+------------------+------------------------+-------+------------------+--------------------+--------------------+--------------+----------+
|_c0|Glucose|               BMI|DiabetesPedigreeFunction|Outcome|Pregnancies_normed|BloodPressure_normed|SkinThickness_normed|Insulin_normed|Age_normed|
+---+-------+------------------+------------------------+-------+------------------+--------------------+--------------------+--------------+----------+
|  0|    161|1.2884700711337882|       1.738458570845793|      0|             0.471|               0.951|               0.606|         0.738|       0.1|
|  1|     34| 16.69615934876864|       0.992257194204867|      0|               0.0|               0.918|                0.03|         0.686|      0.55|
|  2|    109|11.949503547147923|      1.7382799665273752|      0|             0.471|               0.369|                0.01|         0.407|     0.867|
|  3|     81| 43.84995589823813|      1.6192323967547844|      1|             0.05

24/06/10 20:24:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Glucose, BMI, DiabetesPedigreeFunction, Outcome, Pregnancies_normed, BloodPressure_normed, SkinThickness_normed, Insulin_normed, Age_normed
 Schema: _c0, Glucose, BMI, DiabetesPedigreeFunction, Outcome, Pregnancies_normed, BloodPressure_normed, SkinThickness_normed, Insulin_normed, Age_normed
Expected: _c0 but found: 
CSV file: file:///home/nampq/projects/feature-store/batch_processing/test.csv


In [6]:
data \
    .write \
    .mode("overwrite") \
    .parquet(f"s3a://{bucket}/coffee_sales.parquet")

24/06/10 20:24:13 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/nampq/.cache/pypoetry/virtualenvs/feast-4zXjf9Ge-py3.9/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/nampq/.cache/pypoetry/virtualenvs/feast-4zXjf9Ge-py3.9/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/nampq/.pyenv/versions/3.9.12/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

24/06/10 20:34:32 WARN FileSystem: Failed to initialize fileystem s3a://diabetes/coffee_sales.parquet: org.apache.hadoop.fs.s3a.AWSClientIOException: doesBucketExist on diabetes: com.amazonaws.SdkClientException: Unable to execute HTTP request: Connect to localhost:9001 [localhost/127.0.0.1] failed: Connection refused (Connection refused): Unable to execute HTTP request: Connect to localhost:9001 [localhost/127.0.0.1] failed: Connection refused (Connection refused)
24/06/10 22:03:59 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 4294831 ms exceeds timeout 120000 ms
24/06/10 22:03:59 WARN SparkContext: Killing executors is not supported by current scheduler.
24/06/10 22:04:00 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout